In [35]:
from ietfdata.mailarchive import MailArchive
from dotenv import load_dotenv
import os
import pandas as pd
from IPython.core.display import display, HTML

In [15]:
load_dotenv()

archive = MailArchive()
m_list = archive.mailing_list(os.getenv("IETF_MAIL_LIST"))
messages = m_list.messages()

In [17]:
messages_dict = []
for message in messages:
    messages_dict.append({
        "timestamp" : message.date, 
        "from":message.from_addr,
        "subject":message.subject,
        "content" : message.rfc822_message()
        "content_type" : message.get_content_type()
    })

In [46]:
df = pd.DataFrame(messages_dict)
df.head()

,timestamp,from,subject,content
0,2019-07-29 16:58:23,Victor Vasiliev <vasilvv@google.com>,[Webtransport] WebTransport Mailing List,"[Return-Path, X-Original-To, Delivered-To, Rec..."
1,2019-07-29 23:28:57,Eliot Lear <lear@cisco.com>,Re: [Webtransport] WebTransport Mailing List,"[Return-Path, X-Original-To, Delivered-To, Rec..."
2,2019-07-29 17:32:30,Victor Vasiliev <vasilvv@google.com>,Re: [Webtransport] WebTransport Mailing List,"[Return-Path, X-Original-To, Delivered-To, Rec..."
3,2019-07-30 03:53:32,"""Martin J. Dürst"" <duerst@it.aoyama.ac.jp>",Re: [Webtransport] [hybi] WebTransport Mailing...,"[Return-Path, X-Original-To, Delivered-To, Rec..."
4,2019-07-30 09:58:11,Lennart Grahl <lennart.grahl@gmail.com>,Re: [Webtransport] [hybi] WebTransport Mailing...,"[Return-Path, X-Original-To, Delivered-To, Rec..."


In [70]:
df['content_type'] = df['content'].apply(lambda msg: msg.get_content_type())

In [77]:
def get_content_types(msg):
    all_types = []
    if msg.is_multipart():
        for sub_msg in msg.get_payload():
            all_types.append(get_content_types(sub_msg))
    else:
        all_types.append(msg.get_content_type())
    return all_types

types = []
for idx, msg in df['content'].items():
    types.append(get_content_types(msg))

In [78]:
types

[[['text/plain'], ['text/html']],
 [[['text/plain'], ['text/html']], ['application/pgp-signature']],
 [['text/plain'], ['text/html']],
 ['text/plain'],
 ['text/plain'],
 [['text/plain'], ['text/html']],
 ['text/plain'],
 [['text/plain'], ['text/html']],
 [['text/plain'], ['text/html']],
 ['text/plain'],
 [['text/plain'], ['text/html']],
 [['text/plain'], ['text/html']],
 ['text/plain'],
 ['text/plain'],
 [['text/plain'], ['text/html']],
 [['text/plain'], ['text/html']],
 [['text/plain'], ['text/html']],
 ['text/plain'],
 [['text/plain'], ['text/html']],
 [['text/plain'], ['text/html']],
 [['text/plain'], ['text/html']],
 ['text/plain'],
 [['text/plain'], ['text/html']],
 [['text/plain'], ['text/html']],
 [['text/plain'], ['text/html']],
 [['text/plain'], ['text/html']],
 [['text/plain'], ['text/html']],
 [['text/plain'], ['text/html']],
 ['text/plain'],
 [['text/plain'], ['text/html']],
 [['text/plain'], ['text/html']],
 [['text/plain'], ['text/html']],
 [['text/plain'], ['text/html']]